# Nearly-Gaussian Distributions

In [1]:
import sympy as sp
from symlib.random import ExpVal, RandomIndexedBase
from symlib.gaussian import wick_contraction, GaussianIndexedBase, GaussianExpVal
from symlib.tensor import SymmetryIndexedBase, canonicalize_dummy_indices
from symlib.utils import wilds, wild_subs, pull_sums_out_front

we have distribution 
$$
p(z) \equiv \frac{e^{-S(z)}}{Z}
$$
where $S(z)$ is action and $Z$ is partition function, we consider model:
$$
S(z) = \frac{1}{2} \sum_{\mu, \nu = 1}^{N} K^{\mu \nu} z_\mu z_\nu 
+ \frac{\epsilon}{4!} \sum_{\mu, \nu, \rho, \lambda = 1}^{N} 
V^{\mu \nu \rho \lambda} z_\mu z_\nu z_\rho z_\lambda
$$
where the second term is a small quartic_action

In [2]:
K = SymmetryIndexedBase('K', symmetries='full')
V = SymmetryIndexedBase('V', symmetries='full')
mu = sp.symbols('mu1:5', integer = True)
rho = sp.symbols('rho1:5', integer = True)
eps = sp.Symbol('epsilon')
z = GaussianIndexedBase('z')
N = sp.Symbol('N', integer = True)

In [3]:
Z0 = sp.Symbol('\sqrt{|2\pi K|}') #sp.sqrt(sp.Abs(2*sp.pi*K), evaluate=False)
Z0 # partition function for quadratic action

\sqrt{|2\pi K|}

In [4]:
quartic_action = - eps * sp.Sum(V[*rho]*z[rho[0]]*z[rho[1]]*z[rho[2]]*z[rho[3]]/24, 
                (rho[0], 1, N),
                (rho[1], 1, N),
                (rho[2], 1, N),
                (rho[3], 1, N),
                )
quartic_action

-epsilon*Sum(V[rho1, rho2, rho3, rho4]*z[rho1]*z[rho2]*z[rho3]*z[rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))

## Partition Function
$$
Z = \int \left[ \prod_{\mu} dz_\mu \right] e^{-S(z)} \\
= \int \left[ \prod_{\mu} dz_\mu \right] \exp \left( 
    -\frac{1}{2} \sum_{\mu, \nu} K^{\mu \nu} z_\mu z_\nu 
    - \frac{\epsilon}{24} \sum_{\rho_1, \ldots, \rho_4} 
    V^{\rho_1 \rho_2 \rho_3 \rho_4} z_{\rho_1} z_{\rho_2} z_{\rho_3} z_{\rho_4}
\right) \\
= \sqrt{|2 \pi K|} \left\langle \exp \left(
    - \frac{\epsilon}{24} \sum_{\rho_1, \ldots, \rho_4} 
    V^{\rho_1 \rho_2 \rho_3 \rho_4} z_{\rho_1} z_{\rho_2} z_{\rho_3} z_{\rho_4}
\right) \right\rangle_K .
$$
let's work out $Z$:

In [5]:
EK = GaussianExpVal(K)
Z = Z0 * EK(sp.series(sp.exp(quartic_action), x=eps, x0=0, n=2).removeO())
Z

\sqrt{|2\pi K|}*(-epsilon*Sum(⟨z[rho1]*z[rho2]*z[rho3]*z[rho4]⟩_K*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + 1)

In [6]:
Z = wick_contraction(Z)
Z

\sqrt{|2\pi K|}*(-epsilon*Sum((⟨z[rho1]*z[rho2]⟩_K*⟨z[rho3]*z[rho4]⟩_K + ⟨z[rho1]*z[rho3]⟩_K*⟨z[rho2]*z[rho4]⟩_K + ⟨z[rho1]*z[rho4]⟩_K*⟨z[rho2]*z[rho3]⟩_K)*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + 1)

In [7]:
A, B = wilds('A, B')
gaussian_rule = {EK(z[A] * z[B]):K[A, B]}
Z = wild_subs(Z, gaussian_rule)
Z

\sqrt{|2\pi K|}*(-epsilon*Sum((K[rho1, rho2]*K[rho3, rho4] + K[rho1, rho3]*K[rho2, rho4] + K[rho1, rho4]*K[rho2, rho3])*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + 1)

assume full symmetry of $K$ and $V$, so we can sort the index:

In [8]:
Z = pull_sums_out_front(sp.expand(Z))
Z

\sqrt{|2\pi K|} + Sum(-\sqrt{|2\pi K|}*epsilon*K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-\sqrt{|2\pi K|}*epsilon*K[rho1, rho3]*K[rho2, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-\sqrt{|2\pi K|}*epsilon*K[rho1, rho4]*K[rho2, rho3]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))

In [9]:
Z = canonicalize_dummy_indices(Z).simplify()
Z

-\sqrt{|2\pi K|}*epsilon*Sum(K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))/8 + \sqrt{|2\pi K|}

# Two point correlator

$$
\mathbb{E}[z_{\mu_1} z_{\mu_2}] 
= \frac{1}{Z} \int \left[ \prod_{\mu} dz_\mu \right] 
e^{-S(z)} z_{\mu_1} z_{\mu_2} \\
= \frac{\sqrt{2\pi K}}{Z} \left\langle 
z_{\mu_1} z_{\mu_2} 
\exp\left( 
    -\frac{\epsilon}{24} 
    \sum_{\rho_1, \ldots, \rho_4} 
    V^{\rho_1 \rho_2 \rho_3 \rho_4} 
    z_{\rho_1} z_{\rho_2} z_{\rho_3} z_{\rho_4}
\right) 
\right\rangle_K
$$


In [10]:
inner = z[mu[0]]*z[mu[1]]*sp.exp(quartic_action)
limit = sp.series(inner, x=eps, x0=0, n=2).removeO()
limit = pull_sums_out_front(limit)
Ezz = Z0/Z * EK(limit)
Ezz = sp.simplify(Ezz)
Ezz

(epsilon*Sum(⟨z[mu1]*z[mu2]*z[rho1]*z[rho2]*z[rho3]*z[rho4]⟩_K*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) - 24*⟨z[mu1]*z[mu2]⟩_K)/(3*epsilon*Sum(K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) - 24)

In [11]:
# repeat pipeline: take limit -> wick -> wild_subs <z1z2>_K to K12 -> pull sum -> consider symmetry dummy indices
Ezz = sp.series(Ezz, x=eps, x0=0, n=2).removeO()
Ezz = wick_contraction(Ezz)
Ezz = wild_subs(Ezz, gaussian_rule)
Ezz = pull_sums_out_front(sp.expand(sp.simplify(Ezz)))
Ezz = canonicalize_dummy_indices(Ezz).simplify()
Ezz

-epsilon*Sum(K[mu1, rho1]*K[mu2, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))/2 + K[mu1, mu2]

In [22]:
# helper function that will be used in 4-pt corr
def Ez2(i: int, j: int):
    a, b = mu[i-1], mu[j-1]
    return K[a, b] - eps * sp.Sum(V[*rho]*K[a, rho[0]]*K[b, rho[1]]*K[rho[2], rho[3]]/2, 
                (rho[0], 1, N),
                (rho[1], 1, N),
                (rho[2], 1, N),
                (rho[3], 1, N),
                )
Ez2(1,2)

-epsilon*Sum(K[mu1, rho1]*K[mu2, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/2, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + K[mu1, mu2]

# Four point connected correlator

$$ \mathbb{E}\left[{z}_{\mu_{1}} {z}_{\mu_{2}} {z}_{\mu_{3}} {z}_{\mu_{4}}\right] |_{connected} = \mathbb{E}\left[{z}_{\mu_{1}} {z}_{\mu_{2}} {z}_{\mu_{3}} {z}_{\mu_{4}}\right] - \mathbb{E}\left[{z}_{\mu_{1}} {z}_{\mu_{2}}\right] \mathbb{E}\left[{z}_{\mu_{3}} {z}_{\mu_{4}}\right] - \mathbb{E}\left[{z}_{\mu_{1}} {z}_{\mu_{3}}\right] \mathbb{E}\left[{z}_{\mu_{2}} {z}_{\mu_{4}}\right] - \mathbb{E}\left[{z}_{\mu_{1}} {z}_{\mu_{4}}\right] \mathbb{E}\left[{z}_{\mu_{2}} {z}_{\mu_{3}}\right]
$$

In [15]:
inner = z[mu[0]]*z[mu[1]]*z[mu[2]]*z[mu[3]]*sp.exp(quartic_action)
limit = sp.series(inner, x=eps, x0=0, n=2).removeO()
limit = pull_sums_out_front(limit)
coef = sp.series(Z0/Z, x=eps, x0=0, n=2).removeO()
Ez4: sp.Expr = coef * EK(limit)
Ez4

(epsilon*Sum(K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))/8 + 1)*(⟨z[mu1]*z[mu2]*z[mu3]*z[mu4]⟩_K + Sum(-epsilon*⟨z[mu1]*z[mu2]*z[mu3]*z[mu4]*z[rho1]*z[rho2]*z[rho3]*z[rho4]⟩_K*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)))

In [16]:
Ez4 = Ez4.replace(lambda e: isinstance(e, sp.Sum) and isinstance(e.function, sp.Mul) and eps in e.function.args,
        lambda e: eps * sp.Sum(
        sp.Mul(*[arg for arg in e.function.args if arg != eps]),
        *e.limits)
    ) # pull eps out of sum
Ez4 = sp.series(Ez4, x=eps, x0=0, n=2).removeO()
Ez4 = wick_contraction(Ez4)
Ez4 = wild_subs(Ez4, gaussian_rule)
Ez4

epsilon*((K[mu1, mu2]*K[mu3, mu4] + K[mu1, mu3]*K[mu2, mu4] + K[mu1, mu4]*K[mu2, mu3])*Sum(K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))/8 + Sum(-(K[mu1, mu2]*K[mu3, mu4]*K[rho1, rho2]*K[rho3, rho4] + K[mu1, mu2]*K[mu3, mu4]*K[rho1, rho3]*K[rho2, rho4] + K[mu1, mu2]*K[mu3, mu4]*K[rho1, rho4]*K[rho2, rho3] + K[mu1, mu2]*K[mu3, rho1]*K[mu4, rho2]*K[rho3, rho4] + K[mu1, mu2]*K[mu3, rho1]*K[mu4, rho3]*K[rho2, rho4] + K[mu1, mu2]*K[mu3, rho1]*K[mu4, rho4]*K[rho2, rho3] + K[mu1, mu2]*K[mu3, rho2]*K[mu4, rho1]*K[rho3, rho4] + K[mu1, mu2]*K[mu3, rho2]*K[mu4, rho3]*K[rho1, rho4] + K[mu1, mu2]*K[mu3, rho2]*K[mu4, rho4]*K[rho1, rho3] + K[mu1, mu2]*K[mu3, rho3]*K[mu4, rho1]*K[rho2, rho4] + K[mu1, mu2]*K[mu3, rho3]*K[mu4, rho2]*K[rho1, rho4] + K[mu1, mu2]*K[mu3, rho3]*K[mu4, rho4]*K[rho1, rho2] + K[mu1, mu2]*K[mu3, rho4]*K[mu4, rho1]*K[rho2, rho3] + K[mu1, mu2]*K[mu3, rho4]*K[mu4, rho2]*K[rho1, rho3] + K[mu1, mu2]*K[mu3, rho4]*K[mu4, 

In [26]:
conn4 = Ez4 - Ez2(1,2)*Ez2(3,4) - Ez2(1,3)*Ez2(2,4) - Ez2(1,4)*Ez2(2,3)
conn4 = sp.series(conn4, x=eps, x0=0, n=2).removeO()
conn4

epsilon*((K[mu1, mu2]*K[mu3, mu4] + K[mu1, mu3]*K[mu2, mu4] + K[mu1, mu4]*K[mu2, mu3])*Sum(K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))/8 + Sum(-(K[mu1, mu2]*K[mu3, mu4]*K[rho1, rho2]*K[rho3, rho4] + K[mu1, mu2]*K[mu3, mu4]*K[rho1, rho3]*K[rho2, rho4] + K[mu1, mu2]*K[mu3, mu4]*K[rho1, rho4]*K[rho2, rho3] + K[mu1, mu2]*K[mu3, rho1]*K[mu4, rho2]*K[rho3, rho4] + K[mu1, mu2]*K[mu3, rho1]*K[mu4, rho3]*K[rho2, rho4] + K[mu1, mu2]*K[mu3, rho1]*K[mu4, rho4]*K[rho2, rho3] + K[mu1, mu2]*K[mu3, rho2]*K[mu4, rho1]*K[rho3, rho4] + K[mu1, mu2]*K[mu3, rho2]*K[mu4, rho3]*K[rho1, rho4] + K[mu1, mu2]*K[mu3, rho2]*K[mu4, rho4]*K[rho1, rho3] + K[mu1, mu2]*K[mu3, rho3]*K[mu4, rho1]*K[rho2, rho4] + K[mu1, mu2]*K[mu3, rho3]*K[mu4, rho2]*K[rho1, rho4] + K[mu1, mu2]*K[mu3, rho3]*K[mu4, rho4]*K[rho1, rho2] + K[mu1, mu2]*K[mu3, rho4]*K[mu4, rho1]*K[rho2, rho3] + K[mu1, mu2]*K[mu3, rho4]*K[mu4, rho2]*K[rho1, rho3] + K[mu1, mu2]*K[mu3, rho4]*K[mu4, 

In [33]:
conn4 = pull_sums_out_front(sp.expand(conn4))
conn4 = canonicalize_dummy_indices(conn4)
conn4

3*Sum(-epsilon*K[mu1, mu2]*K[mu3, mu4]*K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(epsilon*K[mu1, mu2]*K[mu3, mu4]*K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/8, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + 12*Sum(-epsilon*K[mu1, mu2]*K[mu3, rho1]*K[mu4, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(epsilon*K[mu1, mu2]*K[mu3, rho1]*K[mu4, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/2, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + 3*Sum(-epsilon*K[mu1, mu3]*K[mu2, mu4]*K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(epsilon*K[mu1, mu3]*K[mu2, mu4]*K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/8, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + 12*Sum(-epsilon*K[mu1, mu3]*K[mu2, rho1]*K[mu4, rho2]*K[rho3, rho4]*V[rho1, rho2,

In [34]:
sp.simplify(conn4)

epsilon*Sum(-K[mu1, rho1]*K[mu2, rho2]*K[mu3, rho3]*K[mu4, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))